<a href="https://colab.research.google.com/github/dfragos/EMOS-2024/blob/main/Spacial/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install R and IRkernel
!apt-get install r-base
!R -e "install.packages('IRkernel', repos='http://cran.us.r-project.org')"
!R -e "IRkernel::installspec(user = FALSE)"

Colab menu: Runtime > Change runtime type.
Runtime type: R.

In [ ]:
# Install necessary packages if not already installed
if (!requireNamespace("giscoR", quietly = TRUE)) install.packages("giscoR")
if (!requireNamespace("eurostat", quietly = TRUE)) install.packages("eurostat")
if (!requireNamespace("sf", quietly = TRUE)) install.packages("sf")
if (!requireNamespace("ggplot2", quietly = TRUE)) install.packages("ggplot2")
if (!requireNamespace("dplyr", quietly = TRUE)) install.packages("dplyr")

# Load libraries
library(giscoR)
library(eurostat)
library(sf)
library(ggplot2)
library(dplyr)

In [ ]:
# Search for unemployment data in Eurostat
datasets <- search_eurostat("unemployment")
print(datasets)

# Get the dataset ID for monthly unemployment rates
dataset_id <- "une_rt_m"

# Load unemployment data
unemployment_data <- get_eurostat(id = dataset_id, time_format = "date")

In [ ]:
# Filter data for the latest available month and only total population
latest_date <- max(unemployment_data$TIME_PERIOD)

filtered_data <- unemployment_data %>%
  filter(TIME_PERIOD == latest_date & sex == "T" & age == "TOTAL" & unit == "PC_ACT" ) %>%
  select(geo, values)

# Get spatial data for European countries using giscoR
europe_map <- gisco_get_countries(resolution = "10", year = 2020)

# Filter the map to keep only European countries
europe_map <- europe_map %>%
  filter(EU_STAT == "T")

In [ ]:
# Define a mapping from ISO2 to ISO3
iso2_to_iso3 <- c(
  "AT" = "AUT", "BE" = "BEL", "BG" = "BGR", "CY" = "CYP", "CZ" = "CZE",
  "DE" = "DEU", "DK" = "DNK", "EE" = "EST", "EL" = "GRC", "ES" = "ESP",
  "FI" = "FIN", "FR" = "FRA", "HR" = "HRV", "HU" = "HUN", "IE" = "IRL",
  "IT" = "ITA", "LT" = "LTU", "LU" = "LUX", "LV" = "LVA", "MT" = "MLT",
  "NL" = "NLD", "PL" = "POL", "PT" = "PRT", "RO" = "ROU", "SE" = "SWE",
  "SI" = "SVN", "SK" = "SVK"
)

# Filter out non-country codes from the unemployment data
filtered_data <- filtered_data %>%
  filter(geo %in% names(iso2_to_iso3))

# Add the corresponding ISO3 codes to the unemployment data
filtered_data <- filtered_data %>%
  mutate(ISO3_CODE = iso2_to_iso3[geo])

# Join the spatial data with unemployment data using ISO3 codes
europe_unemployment <- europe_map %>%
  left_join(filtered_data, by = "ISO3_CODE")

In [ ]:
# Use eurostat
library(eurostat)
popdens <- get_eurostat("demo_r_d3dens") %>%
  filter(TIME_PERIOD == "2021-01-01")

In [ ]:
# Check for missing data
missing_data <- europe_unemployment %>%
  filter(is.na(values))

if (nrow(missing_data) > 0) {
  print("Some countries are missing unemployment rates:")
  print(missing_data)
}

In [ ]:
# Create the choropleth map
ggplot(europe_unemployment) +
  geom_sf(aes(fill = values), color = "black", size = 0.1) +
  scale_fill_viridis_c(option = "plasma", name = "Unemployment Rate (%)") +
  labs(
    title = "Unemployment Rates in Europe",
    subtitle = paste("Latest data as of", format(latest_date, "%B %Y")),
    caption = "Source: Eurostat and GISCO"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 16, face = "bold"),
    plot.subtitle = element_text(size = 12),
    legend.position = "right"
  )
